In [1]:
import opendatasets as od

In [2]:
od.download('https://www.kaggle.com/c/quora-question-pairs/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shekolla
Your Kaggle Key: ········


100%|████████████████████████████████████████| 309M/309M [00:35<00:00, 9.25MB/s]



Extracting archive ./quora-question-pairs/quora-question-pairs.zip to ./quora-question-pairs


In [4]:
!ls quora-question-pairs/

sample_submission.csv.zip test.csv.zip              train.csv.zip
test.csv                  train.csv


In [5]:
import pandas as pd
import numpy as np

In [73]:
df = pd.read_csv('quora-question-pairs/train.csv', index_col='id')

In [74]:
df.sample(10)


,qid1,qid2,question1,question2,is_duplicate
id,,,,,
60101,36634,28740,Is it possible to love more than one person at...,Can you love two people at the same time?,1
150166,236399,236400,My girlfriend's parents found out about our re...,Haryanvi best qualities?,0
48268,86093,86094,Yoga: What is the right way to do Kapalbhati P...,Why don't Hindus learn Sanskrit and yoga when ...,0
133781,213963,16100,"How can I earn doing freelancing in India, Mum...",What shall I do to become a successful freelan...,0
314795,439589,439590,How can we score good bands in ielts writing?,How can we score good bands in ielts reading?,0
119421,193910,193911,How do women feel when pussy is licked?,Is pussy licking good for health?,0
144498,228675,83802,What are Hillary Clinton's views on the Israel...,What is Hillary Clinton's plan to make peace b...,1
193857,293896,293897,What is the best name for the business related...,AREA NEEDED for KEEPING 100 BROILER?,0
360750,425415,444670,What happens when you mix hydrochloric acid wi...,We think the acid reflux problem is due to hyd...,0


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404290 entries, 0 to 404289
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   qid1          404290 non-null  int64 
 1   qid2          404290 non-null  int64 
 2   question1     404289 non-null  object
 3   question2     404288 non-null  object
 4   is_duplicate  404290 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 18.5+ MB


In [76]:
from sklearn.feature_extraction.text import CountVectorizer

In [77]:
cv = CountVectorizer(max_features=3000)

def return_vector(df1):
    combined_li = list(df1['question1'].apply(lambda x: str(x).lower())) + list(df1['question2'].apply(lambda x: str(x).lower()))
    cv_ = cv.fit_transform(combined_li).toarray()
    print(cv_.shape)
    cv_ = np.vsplit(cv_, 2)
    return cv_
    

In [113]:
samp = df
print(samp.shape)
lo = return_vector(samp)

(404290, 5)
(808580, 3000)


In [114]:
new_df = pd.DataFrame(np.hstack(lo))

In [115]:
y = samp.iloc[:, -1]

In [116]:
y.shape

(404290,)

In [117]:
from sklearn.model_selection import train_test_split

In [118]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y)

In [119]:
from sklearn.naive_bayes import MultinomialNB

In [120]:
mnb = MultinomialNB()

In [ ]:
mnb.fit(X_train, y_train)

In [ ]:
y_pred = mnb.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
print(classification_report(y_pred, y_test))
print(accuracy_score(y_pred, y_test))

In [101]:
from sklearn.ensemble import RandomForestClassifier

In [104]:
rfc = RandomForestClassifier()

In [105]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [106]:
y_pred = rfc.predict(X_test)

In [107]:
print(classification_report(y_pred, y_test))
print(accuracy_score(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.91      0.74      0.81      5822
           1       0.45      0.76      0.56      1678

    accuracy                           0.74      7500
   macro avg       0.68      0.75      0.69      7500
weighted avg       0.81      0.74      0.76      7500

0.7396


In [108]:
from xgboost import XGBClassifier

In [109]:
xgb_c = XGBClassifier()

In [110]:
xgb_c.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [111]:
y_pred = xgb_c.predict(X_test)

In [112]:
print(classification_report(y_pred, y_test))
print(accuracy_score(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.91      0.72      0.80      5910
           1       0.41      0.73      0.53      1590

    accuracy                           0.72      7500
   macro avg       0.66      0.73      0.67      7500
weighted avg       0.80      0.72      0.75      7500

0.7228
